In [20]:
import pandas as pd

In [21]:
rules = pd.read_json("../data/arules.json")

In [22]:
rules["antecedents"] = rules["antecedents"].apply(lambda x: frozenset(x))
rules["consequents"] = rules["consequents"].apply(lambda x: frozenset(x))

In [23]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(CryptoCurrency),(movies),0.035937,0.083617,0.003288,0.091503,1.094312,0.000283,1.008680
1,(movies),(CryptoCurrency),0.083617,0.035937,0.003288,0.039326,1.094312,0.000283,1.003528
2,(CryptoCurrency),(AmItheAsshole),0.035937,0.038403,0.000470,0.013072,0.340389,-0.000910,0.974334
3,(AmItheAsshole),(CryptoCurrency),0.038403,0.035937,0.000470,0.012232,0.340389,-0.000910,0.976002
4,(nba),(CryptoCurrency),0.026307,0.035937,0.001644,0.062500,1.739175,0.000699,1.028334
...,...,...,...,...,...,...,...,...,...
2934205,(technology),"(Wellthatsucks, science, SelfAwarewolves, movi...",0.051908,0.000235,0.000235,0.004525,19.264706,0.000223,1.004310
2934206,(Showerthoughts),"(Wellthatsucks, science, SelfAwarewolves, movi...",0.082443,0.000235,0.000235,0.002849,12.129630,0.000216,1.002622
2934207,(PS5),"(Wellthatsucks, science, SelfAwarewolves, movi...",0.017264,0.000235,0.000235,0.013605,57.925170,0.000231,1.013555
2934208,(science),"(Wellthatsucks, SelfAwarewolves, movies, facep...",0.060129,0.000235,0.000235,0.003906,16.630859,0.000221,1.003686


In [24]:
for column in ["support", "confidence", "lift", "leverage", "conviction"]:
#     rules[column] = (rules[column] - rules[column].mean())/rules[column].std()
    rules[column] = (rules[column] - rules[column].min())/(rules[column].max() - rules[column].min())

In [25]:
# OBVIOUS_SUBREDDITS = {'reddit.com', 'AskReddit', 'pics', 'WTF', 'IAmA'}

In [26]:
def get_rules(set_of_subreddits):
    antecedents_rule =  rules['antecedents'].apply(lambda x: set_of_subreddits.issuperset(x))
    new_rules =  rules[antecedents_rule].copy()
#     new_rules["Coefficient"] = new_rules["confidence"] + new_rules["support"]
#     new_rules["Coefficient"] = new_rules["lift"]
    new_rules["Coefficient"] = new_rules["confidence"] - new_rules["support"] + new_rules["lift"] + new_rules["leverage"]+ new_rules["conviction"]
#     new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits - OBVIOUS_SUBREDDITS)
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits)
    new_rules = new_rules[new_rules["consequents"].apply(lambda x: len(x) > 0)]
    if len(new_rules) == 0:
        return []
    new_rules = new_rules[['consequents', "Coefficient"]]
    new_rules = new_rules.explode("consequents")
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x:list(x)[0])
    new_rules = new_rules.groupby("consequents")["Coefficient"].max().reset_index()    
    return list(new_rules.nlargest(100, "Coefficient")["consequents"])

In [32]:
df = get_rules({ 'DestinyTheGame', 'Diablo', 'CitiesSkylines', 'CODWarzone', 'zelda', 'witcher'})
df

['destiny2',
 'movies',
 'YouShouldKnow',
 'gifs',
 'technology',
 'WhitePeopleTwitter',
 'IdiotsInCars',
 'leagueoflegends',
 'TechNewsToday',
 'Unexpected',
 'PoliticalHumor',
 'Breath_of_the_Wild',
 'blackopscoldwar',
 'Damnthatsinteresting',
 'nottheonion',
 'NintendoSwitch',
 'MaliciousCompliance',
 'MapPorn',
 'oddlysatisfying',
 'Music',
 'soccer',
 'ffxiv',
 'assholedesign',
 'dankmemes',
 'MovieDetails',
 'iamatotalpieceofshit',
 'science',
 'modernwarfare',
 'IAmA',
 'wow',
 'PS5',
 'pcgaming',
 'WTF',
 'cyberpunkgame',
 'MMORPG',
 'mildlyinteresting',
 'dndmemes',
 'me_irl',
 'Terraria',
 'NoStupidQuestions',
 'TrueOffMyChest',
 'tifu',
 'Minecraft',
 'MadeMeSmile',
 'HolUp',
 'Amd',
 'insanepeoplefacebook',
 'pcmasterrace',
 'facepalm',
 'ShittyLifeProTips',
 'Jokes',
 'videos',
 'TwoXChromosomes',
 'pokemon',
 'trees',
 'CrusaderKings',
 'PrequelMemes',
 'Showerthoughts',
 'civ',
 'Cringetopia',
 'WritingPrompts',
 'PS4',
 'AnimalCrossing',
 'DestinyMemes',
 'therewasanatt

In [31]:
df = get_rules({ 'Bitcoin'})
df

['CryptoCurrency',
 'amcstock',
 'stocks',
 'ethtrader',
 'options',
 'technology',
 'binance',
 'ethereum',
 'GME',
 'investing',
 'cardano',
 'pennystocks',
 'CryptoMoonShots',
 'SatoshiStreetBets',
 'Damnthatsinteresting',
 'Wallstreetbetsnew',
 'oddlysatisfying',
 'nba',
 'personalfinance',
 'IdiotsInCars',
 'OldSchoolCool',
 'thetagang',
 'LifeProTips',
 'Conservative',
 'Coronavirus',
 'PoliticalHumor',
 'RobinHoodPennyStocks',
 'StockMarket',
 'coolguides',
 'gifs',
 'technicallythetruth',
 'WallStreetbetsELITE',
 'Buttcoin',
 'TheRightCantMeme',
 'WatchPeopleDieInside',
 'apple',
 'buildapc',
 'TikTokCringe',
 'mildlyinteresting',
 'CanadianInvestor',
 'dataisbeautiful',
 'iamatotalpieceofshit',
 'television',
 'IllegalLifeProTips',
 'MovieDetails',
 'marvelstudios',
 'nvidia',
 'toronto',
 'DataHoarder',
 'Wellthatsucks',
 'movies',
 'MadeMeSmile',
 'science',
 'Economics',
 'Showerthoughts',
 'ActualPublicFreakouts',
 'WhitePeopleTwitter',
 'Whatcouldgowrong',
 'EthereumClass